In [ ]:
! pip install -r requirements.txt --quiet

# Semenatic Kernel and the OpenAPI Plugin


🔗 [Add plugins from OpenAPI specifications](https://learn.microsoft.com/en-us/semantic-kernel/concepts/plugins/adding-openapi-plugins)

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent,ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.openapi_plugin import OpenAPIFunctionExecutionParameters
from dotenv import load_dotenv
from os import environ
import asyncio
import requests

load_dotenv(override=True)


In [ ]:
kernel = Kernel()

kernel.add_service(AzureChatCompletion(
    service_id="chat",
    deployment_name=environ["AZURE_OPENAI_CHAT_MODEL"],
    endpoint=environ["AZURE_OPENAI_ENDPOINT"],
    api_key=environ["AZURE_OPENAI_API_KEY"] ))

kernel.add_plugin_from_openapi(
   plugin_name="sales",
   openapi_document_path=environ['OPENAPI_URL'],
   execution_settings=OpenAPIFunctionExecutionParameters(
         enable_payload_namespacing=True,
   ),
)

In [ ]:
instructions = """You are AutoSales Analyst, an AI agent specialized in analyzing automotive sales data.

Your objectives:
- Dynamically choose and call the appropriate tool(s) to answer user questions about sales, customers, or products.
- Always include customer names alongside IDs in any output.
- Always display monetary amounts in $USD (e.g., $123.45).
- Use filters and aggregations as needed to generate insights from sales orders, products, and customers.
- Compute totals, revenue, discounts, and other metrics from nested order data.
- Return structured, concise results suitable for analysis or reporting.
- Avoid hardcoding analytics; rely on the tools and their parameters.
- Clarify ambiguous queries before performing analysis.
- Treat the tools as the source of truth; do not expose raw database internals.
- Return customer_id and product_id alongside names in all outputs.

Behavioral guidance:
- For customer-related questions, resolve names and IDs before analyzing orders.
- For product-related questions, resolve categories or IDs before analyzing sales.
"""

In [ ]:
agent = ChatCompletionAgent(
    kernel=kernel, 
    name="SalesAgentOpenAPI",
    instructions=instructions
)

response = await agent.get_response(messages="Which customers had the most orders?")
print(response)
